## Create Labels

### Imports

In [ ]:
import argparse
import json
import numpy as np
import torch
from pathlib import Path
from pcst_fast import pcst_fast as pcst
from tqdm import tqdm

### Métodos

Executa múltiplas tentativas do algoritmo PCST (Prize-Collecting Steiner Tree) com diferentes configurações de parâmetros, selecionando a solução que resulta no menor custo total de conexão.

In [ ]:
def call_pcst_ensemble(edges, prizes, costs, root):
    trials = [
        (edges, prizes, costs, root),
        (edges, prizes, costs, root, 1),
        (edges, prizes, costs, root, 1, "gw"),
        (edges, prizes, costs, root, 1, "strong"),
        (edges, prizes, costs, root, 1, "gw", 0),
        (edges, prizes, costs, root, 1, "strong", 0),
    ]
    best_res, best_cost = None, float('inf')
    for args in trials:
        try:
            n, e = pcst(*args)
            c = costs[e].sum() if len(e) > 0 else 0.0
            if c < best_cost:
                best_cost, best_res = c, (n, e)
        except: pass
    
    if best_res is None: raise RuntimeError("PCST falhou")
    return best_res

Consolida as conexões duplicadas em arestas únicas, priorizando o tipo de relação mais forte e atribuindo custos diferentes (1.0 ou 1.2) para cada ligação com base na sua categoria.

In [ ]:
def build_undirected_edges(g):
    ei, et = g["edge_index"], g["edge_type"]
    pair_type = {}
    for i in range(ei.size(1)):
        u, v = int(ei[0, i]), int(ei[1, i])
        t = int(et[i])
        k = tuple(sorted((u, v)))

        if k not in pair_type or (t == 0 and pair_type[k] != 0):
            pair_type[k] = t
            
    edges, costs = [], []
    for (a, b), t in pair_type.items():
        edges.append([a, b])
        costs.append(1.0 if t == 0 else 1.2)
    return np.array(edges, dtype=np.int64), np.array(costs, dtype=np.float64)

### Main

Configuração de caminhos com criação de subpastas automática

In [ ]:
input_dir = Path("../../inputs")
output_dir = Path("../../outputs")

snapshot_dir = output_dir / "snapshots"
snapshot_dir.mkdir(parents=True, exist_ok=True)

instance_dir = output_dir / "instances"
instance_dir.mkdir(parents=True, exist_ok=True)

label_dir = output_dir / "labels"
label_dir.mkdir(parents=True, exist_ok=True)

Verifica se o arquivo que contém as instâncias processadas existe no diretório e interrompe a execução com um erro caso ele não seja localizado.

In [ ]:
instances_file = instance_dir / "instances.jsonl"
if not instances_file.exists(): 
    raise FileNotFoundError("instances.jsonl não encontrado")

Percorre o arquivo de instâncias para contar quantas linhas (registros) existem no total, permitindo exibir a quantidade exata de trabalho que será processado.

In [ ]:
total_lines = sum(1 for _ in open(instances_file, "r"))

Inicializa dicionários vazios para armazenar grafos e conexões na memória temporária (cache), evitando leituras repetidas do disco, e zera o contador de registros processados.

In [ ]:
graph_cache, edge_cache = {}, {}
written = 0

Processa cada instância para carregar o grafo correspondente, calcula a árvore de custo mínimo (PCST) conectando o servidor às antenas ativas e salva o resultado final como um rótulo (label) para o treinamento posterior.

In [ ]:
with open(label_dir / "labels.jsonl", "w", encoding="utf-8") as out_f:
    for line in tqdm(open(instances_file, "r"), total=total_lines, desc="labels: gerando labels", unit="instância"):
        if not line.strip(): 
            continue

        inst = json.loads(line)
        snap = inst.get("snapshot_next") or inst.get("snapshot")
        if snap not in graph_cache:
            graph_cache[snap] = torch.load(snapshot_dir / f"as_graph_{snap}.pt", map_location="cpu")

        g = graph_cache[snap]
        if snap not in edge_cache:
            edge_cache[snap] = build_undirected_edges(g)

        edges, costs = edge_cache[snap]
        prizes = np.zeros(g["num_nodes"]); prizes[inst["root"]] = 10.0
        for t in inst["terminals_out"]: 
            prizes[int(t)] = 10.0

        sel_nodes, sel_e_idx = call_pcst_ensemble(edges, prizes, costs, inst["root"])
        inst.update({
            "tree_nodes": list(map(int, sel_nodes)),
            "tree_edges": edges[sel_e_idx].tolist()
        })
        
        out_f.write(json.dumps(inst, ensure_ascii=False) + "\n")
        written += 1
            
print(f"  -> labels gerados: {written} registros\n")